In [1]:
import os
from nltk.corpus import stopwords
import string
from collections import Counter

In [2]:
os.getcwd()

'C:\\Users\\USX28939\\PYTHON_CODE_BASE\\GitHub_Doc\\TensorFlow_Framework'

In [3]:
os.chdir("C:\\Machine Learning\\Hackathons\\Sample Data\\NLP\\Sentiment-Analysis\\txt_sentoken\\")

In [4]:
def load_doc(filename):

    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
a = load_doc("C:\\Machine Learning\\Hackathons\\Sample Data\\NLP\\Sentiment-Analysis\\txt_sentoken\\\\neg\\cv000_29416.txt")
a

In [ ]:
string.punctuation

In [5]:
# load all docs in a directory
def process_docs(directory, vocab, is_trian):
# walk through all files in the folder
    for filename in os.listdir(directory):
        # skip any reviews in the test set
        if is_trian and filename.startswith('cv9'):
                continue
        if not is_trian and not filename.startswith('cv9'):
                continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)

maketrans function

x - If only one argument is supplied, it must be a dictionary.The dictionary should contain 1-to-1 mapping from a single character string to its translation OR a unicode number (97 for 'a') to its translation.

y - If two arguments are passed, it must be two strings with equal length.Each character in the first string is a replacement to its corresponding index in the second string.

z - If three arguments are passed, each character in the third argument is mapped to None.


In [6]:
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    #print("table values are :\n")
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


In [ ]:
t = clean_doc(a)
t

In [7]:
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

In [8]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs(os.getcwd() +"\\neg" , vocab, True)
process_docs(os.getcwd() + "\\pos", vocab, True)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

44276
[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('bad', 1248), ('could', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]


In [9]:
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print(len(tokens))

25767


In [ ]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

In [ ]:
# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

In [ ]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [ ]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(len(vocab))

## Embedding Layer while Training

In [10]:
# turn a doc into clean tokens
def clean_doc_emb(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

In [11]:
# load all docs in a directory
def process_docs_emb(directory, vocab, is_trian):
    documents = list()
    # walk through all files in the folder
    for filename in os.listdir(directory):
        # skip any reviews in the test set
        if is_trian and filename.startswith('cv9'):
            continue
        if not is_trian and not filename.startswith('cv9'):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load the doc
        #print(path)
        doc = load_doc(path)
        # clean doc
        tokens = clean_doc_emb(doc, vocab)
        # add to list
        documents.append(tokens)
    return documents

In [12]:

# load all training reviews
positive_docs = process_docs_emb(os.getcwd()+"/pos", vocab, True)
negative_docs = process_docs_emb(os.getcwd()+"/neg", vocab, True)
train_docs = negative_docs + positive_docs

In [13]:
len(train_docs)

1800

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import array

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Tokenizer develops a vocabulary of all tokens in the training dataset and develops a consistent mapping from words in the vocabulary to unique integers. 

In [15]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)

In [16]:
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [17]:
len(encoded_docs)

1800

In [18]:
# pad sequences
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [19]:
Xtrain.shape

(1800, 1380)

In [20]:
# define training labels
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])

In [21]:
ytrain.shape

(1800,)

In [22]:
# load all test reviews
positive_docs_test = process_docs_emb(os.getcwd()+"/pos", vocab, False)
negative_docs_test = process_docs_emb(os.getcwd()+"/neg", vocab, False)
test_docs = negative_docs_test + positive_docs_test
# sequence encode
encoded_docs_test = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post')
# define test labels
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LSTM

In [24]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
max_length

In [31]:
# define model
lstm_out = 200
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
#model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
#model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='softmax'))
print(model.summary())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1380, 100)         4427700   
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               240800    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 4,668,701
Trainable params: 4,668,701
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=5, verbose=2)

Epoch 1/5
 - 537s - loss: 7.9712 - acc: 0.5000
Epoch 2/5
 - 532s - loss: 7.9712 - acc: 0.5000
Epoch 3/5
 - 529s - loss: 7.9712 - acc: 0.5000
Epoch 4/5
 - 522s - loss: 7.9712 - acc: 0.5000
Epoch 5/5
 - 545s - loss: 7.9712 - acc: 0.5000


In [30]:
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: {} and loss is {}' .format(acc*100,loss))

Test Accuracy: 50.0 and loss is 0.6931787490844726


## Train word2vec Embedding

In [ ]:
# load doc into memory
def load_doc_learn_emb(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc_learn_emb(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [ ]:
# turn a doc into clean tokens
def doc_to_clean_lines_learn_emb(doc, vocab):
    clean_lines = list()
    lines = doc.splitlines()
    for line in lines:
        # split into tokens by white space
        tokens = line.split()
        # remove punctuation from each token
        table = str.maketrans('', '', string.punctuation)
        tokens = [w.translate(table) for w in tokens]
        # filter out tokens not in vocab
        tokens = [w for w in tokens if w in vocab]
        clean_lines.append(tokens)
    return clean_lines

In [ ]:
def process_docs_learn_emb(directory, vocab, is_trian):
    lines = list()
    # walk through all files in the folder
    for filename in os.listdir(directory):
        # skip any reviews in the test set
        if is_trian and filename.startswith('cv9'):
            continue
        if not is_trian and not filename.startswith('cv9'):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        doc = load_doc_learn_emb(path)
        doc_lines = doc_to_clean_lines_learn_emb(doc, vocab)
        # add lines to list
        lines += doc_lines
    return lines


In [ ]:
# load training data
positive_lines_learn_emb = process_docs_learn_emb(os.getcwd()+"/pos", vocab, True)
negative_lines_learn_emb = process_docs_learn_emb(os.getcwd()+"/neg", vocab, True)
sentences_learn_emb = negative_lines_learn_emb + positive_lines_learn_emb
print('Total training sentences: %d' % len(sentences_learn_emb))

In [ ]:
len(sentences_learn_emb)

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# train word2vec model
model = Word2Vec(sentences_learn_emb, size=100, window=5, workers=8, min_count=1)
# summarize vocabulary size in model
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

In [ ]:
model.wv.word_vec

In [ ]:
# save model in ASCII (word2vec) format
filename = 'embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [ ]:
model.wv.word_vec("film")

In [ ]:
model.wv.similarity("movie","movie")

In [ ]:
from numpy import asarray
from numpy import zeros

In [ ]:
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [ ]:
t = load_embedding("embedding_word2vec.txt")

In [ ]:
tokenizer.word_index

In [ ]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

In [ ]:
# load embedding from file
raw_embedding = load_embedding('embedding_word2vec.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=True)

In [ ]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

In [ ]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: {} and loss is {}' .format(acc*100,loss))

# Using Pre Trained Layer

In [ ]:
# load embedding as a dict
def load_embedding(filename):
    # load embedding into memory, skip first line
    file = open(filename,'r',encoding="utf8")
    lines = file.readlines()
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
        # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [ ]:
def get_weight_matrix(embedding, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        vector = embedding.get(word)
        if vector is not None:
            weight_matrix[i] = vector
    return weight_matrix

In [ ]:
# load embedding from file
raw_embedding = load_embedding('glove.6B.100d.txt')
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], input_length=max_length, trainable=False)

In [ ]:
# define model
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
print(model.summary())


In [ ]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=7, verbose=2)


In [ ]:
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: {} and loss is {}' .format(acc*100,loss))

In [ ]:
strr = [["message","Two"],["message","one","is"],["This","is","message","three"]]

In [ ]:
t = Tokenizer()

In [ ]:
t.fit_on_texts(strr)

In [ ]:
t.word_index

In [ ]:
t.index_word

In [ ]:
t.index_docs

In [ ]:
t.word_docs

In [ ]:
for w,c in list(t.word_docs.items()):
    print(w, "   ", c)